In [1]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import math
import numpy as np
from math import sqrt
from torch import nn
from torch.nn import init
from torch.nn import functional as F
from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange

In [3]:
# Here's MLP scoring test
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.leakyrelu = nn.LeakyReLU(0.2)
        # self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x1, x2, max_len):
        cosine_score = F.cosine_similarity(x1, x2)
        tensor_similarity = torch.Tensor([[cosine_score]])
        tensor_max_len = torch.Tensor([[max_len]])

        tensor_similarity.requires_grad_(True)
        tensor_max_len.requires_grad_(True)

        x = torch.cat((x1, x2, tensor_similarity, tensor_max_len), dim=1)
        x = self.fc1(x)
        x = self.leakyrelu(x)
        x = self.fc2(x)

        x = torch.abs(x)

        return x

# # 创建并使用MLP模型
# input_dim = 50*2  # 连接后的向量维度
# hidden_dim = 128
# output_dim = 1  # 输出一个分数
# mlp = MLP(input_dim, hidden_dim, output_dim)

# # 生成示例输入向量
# vector1 = torch.randn(1, 50)  # 第一个向量
# vector2 = torch.randn(1, 50)  # 第二个向量

# # 将输入向量传递给MLP模型
# # score = mlp(vector1, vector2)

# # print('This is the score', score)  # 输出分数张量

### Training MLP vs cosine similarity

In [ ]:
# Test cosine similarity with MLP

# 创建并使用MLP模型
input_dim = 50*2  # 连接后的向量维度
hidden_dim = 128
output_dim = 1  # 输出一个分数
mlp = MLP(input_dim, hidden_dim, output_dim)

# build 3 testing tensor
tensor1 = torch.randn(1, 50)
tensor2 = tensor1.clone()  # 通过克隆方式创建一个与tensor1相同的张量
tensor3 = torch.randn(1, 50) # 创建一个不相似的张量

# 将两个向量连接成一个输入向量
input_vector_1 = torch.cat((tensor1, tensor2), dim=1)  # 2 similar vectors
input_vector_2 = torch.cat((tensor1, tensor3), dim=1)  # 2 non-similar vectors

print(input_vector_1)
print(input_vector_2)

# 将输入向量传递给MLP模型
score_1 = mlp(tensor1, tensor2)
score_2 = mlp(tensor1, tensor3)

print('This is the score-1', score_1)  # 输出分数张量
print('This is the score-2', score_2)  # 输出分数张量


similarity_score_1 = F.cosine_similarity(tensor1, tensor2)
similarity_score_2 = F.cosine_similarity(tensor1, tensor3)

print('This is the cos score-1', similarity_score_1)  # 输出分数张量
print('This is the cos score-2', similarity_score_2)  # 输出分数张量

tensor([[ 0.9346, -0.0190, -0.0442, -1.1277, -2.2433,  1.0628,  0.2738, -0.7498,
          1.4525,  0.5115,  1.4996,  0.2842,  0.1637,  1.6438,  0.7192,  1.7105,
          0.7663,  0.1757, -0.3213, -0.1413,  0.4713, -0.1653, -1.0831,  1.7135,
         -0.0699,  0.8162,  0.8200,  0.9509,  1.7064, -1.0233,  1.0895,  0.7483,
         -0.1954, -1.1060, -1.2792, -2.2550, -0.4848,  0.8326,  1.9951, -0.8836,
          0.1315, -0.5436, -0.3903, -1.9608, -0.2627,  0.2624,  1.4792,  0.2979,
          1.4561, -0.0064,  0.9346, -0.0190, -0.0442, -1.1277, -2.2433,  1.0628,
          0.2738, -0.7498,  1.4525,  0.5115,  1.4996,  0.2842,  0.1637,  1.6438,
          0.7192,  1.7105,  0.7663,  0.1757, -0.3213, -0.1413,  0.4713, -0.1653,
         -1.0831,  1.7135, -0.0699,  0.8162,  0.8200,  0.9509,  1.7064, -1.0233,
          1.0895,  0.7483, -0.1954, -1.1060, -1.2792, -2.2550, -0.4848,  0.8326,
          1.9951, -0.8836,  0.1315, -0.5436, -0.3903, -1.9608, -0.2627,  0.2624,
          1.4792,  0.2979,  

### Training MLP test

In [4]:
# Build a Locater with CNN+MLP
''' CNN accept transcript or query and convert it into average embedding, 
    MLP accept query embedding & transcript embedding and convert it into a 2-D vector as <start, end> of relevant spans '''

class testCNNModel(nn.Module):
    def __init__(self):
        super(testCNNModel, self).__init__()
        
        self.conv1d = nn.Conv1d(in_channels=512, out_channels=256, kernel_size=1)
        # self.relu = nn.ReLU()
        # self.drop = nn.Dropout(0.2)
        self.maxpool = nn.MaxPool1d(kernel_size=1)
        self.linear = nn.Linear(196608, 50)   # fully-connected layer
    
    def forward(self, x):
        x = x.requires_grad_(True)
        x = self.conv1d(x)
        # x = self.drop(x)
        x = self.maxpool(x)
        x = torch.flatten(x, start_dim=1)  # 将张量展平
        x = self.linear(x)
        return x

class Locater(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.CNN = testCNNModel()
        self.MLP = MLP(input_dim, hidden_dim, output_dim)

    def forward(self, transcript_emb, query_emb, max_len):
        transcript_cnn_out = self.CNN(transcript_emb)
        query_cnn_out = self.CNN(query_emb)

        spans_id = self.MLP(transcript_cnn_out, query_cnn_out, max_len)

        return spans_id


### Training test

In [ ]:
# Test data
transcript_emb = torch.randn(1, 512, 768)  # tensor for transcript
query_emb_1 = torch.randn(1, 512, 768)  # tensor for query
query_emb_2 = torch.randn(1, 512, 768)
query_emb_3 = torch.randn(1, 512, 768)
query_emb_4 = torch.randn(1, 512, 768)

transcript_emb.requires_grad_(True)
query_emb_1.requires_grad_(True)
query_emb_2.requires_grad_(True)
query_emb_3.requires_grad_(True)
query_emb_4.requires_grad_(True)

# Label
label_1 = torch.Tensor([24, 30])
label_2 = torch.Tensor([0, 10])
label_3 = torch.Tensor([24, 26])
label_4 = torch.Tensor([25, 32])

label_1.requires_grad_(True)
label_2.requires_grad_(True)
label_3.requires_grad_(True)
label_4.requires_grad_(True)

query_set = [query_emb_1, query_emb_2, query_emb_3, query_emb_4]
start_end_label = [label_1, label_2, label_3, label_4] #label

# Instantiate Locater
input_dim = 50*2  # Dimension after connected
hidden_dim = 128
output_dim = 2  # output <start, end> spans
locater = Locater(input_dim, hidden_dim, output_dim)


# Loss function
# criterion = nn.CrossEntropyLoss()  # can't work
criterion = nn.MSELoss()  

# Optimizer
from torch import optim
optimizer = optim.Adam(mlp.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
  running_loss = 0.0
  for i, (inputs_query, label) in enumerate(zip(query_set, start_end_label)):
    print('i=',i)
    print('inputs=',inputs_query)
    # 前向传播
    spans = locater(transcript_emb, inputs_query)
    loss = criterion(spans, label)  # 计算loss误差

    print('predicted spans: ', spans)

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    # 打印每个epoch的平均损失
  epoch_loss = running_loss / len(query_set)
  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")
  print('------------------------------------------------------')

i= 0
inputs= tensor([[[-0.2619, -0.6147,  0.7968,  ..., -0.2643,  1.1212,  0.7796],
         [-0.1804, -0.4150, -0.6541,  ...,  0.7962,  1.4538, -1.2213],
         [-1.0690, -0.1759, -0.7373,  ..., -0.1449,  1.4295, -1.4209],
         ...,
         [-0.3537, -0.8999, -1.2690,  ...,  0.8678,  1.3526,  1.7176],
         [ 1.4875,  0.1943, -2.9266,  ...,  1.3069, -0.1743,  0.8847],
         [ 0.1588,  0.0406,  1.1313,  ..., -0.2899, -0.9426,  0.9801]]],
       requires_grad=True)
predicted spans:  tensor([[0., -0.]], grad_fn=<RoundBackward0>)
i= 1
inputs= tensor([[[ 0.4778,  1.3883, -0.8741,  ...,  1.4224,  2.2957, -0.3944],
         [ 2.6525, -0.4972,  1.7572,  ...,  0.0954, -0.4087, -0.5127],
         [-0.6099,  1.3736, -0.4484,  ...,  1.0382, -0.3736, -1.4498],
         ...,
         [ 0.2059,  0.4491, -1.7673,  ..., -0.4939,  0.3272, -0.9960],
         [ 0.8828,  0.0485, -1.8293,  ..., -0.2453,  0.9305, -1.6672],
         [-0.9344,  0.7967, -0.4213,  ...,  0.8275, -0.8511, -1.0107]]],

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


predicted spans:  tensor([[0., -0.]], grad_fn=<RoundBackward0>)
i= 2
inputs= tensor([[[-0.9681,  0.2716,  1.4775,  ...,  1.4349, -0.4828, -2.3926],
         [-1.9622,  1.5685, -0.3171,  ..., -1.3052,  0.7432,  0.2840],
         [-0.3171, -1.1408,  0.8333,  ..., -0.0348,  0.1850,  0.9617],
         ...,
         [-1.1376, -0.0100,  0.7508,  ...,  0.2652,  0.7560, -0.2433],
         [-2.0999,  0.9153, -0.3850,  ...,  0.4209,  0.6160,  0.0331],
         [ 0.0519, -1.1740,  2.9918,  ...,  0.3874, -0.9176,  1.0621]]],
       requires_grad=True)
predicted spans:  tensor([[0., -0.]], grad_fn=<RoundBackward0>)
i= 3
inputs= tensor([[[ 1.4751, -2.2790, -0.5338,  ..., -0.2522, -0.2616, -1.3452],
         [ 0.3894,  1.0400, -0.5621,  ..., -0.7888,  0.2940, -1.3033],
         [-1.3584, -0.2238, -2.6254,  ..., -0.6882, -0.4461,  0.0454],
         ...,
         [-0.2897,  0.7729, -0.4392,  ...,  0.5651, -1.3621, -0.3709],
         [-1.3717, -1.3365, -0.6513,  ..., -0.4769,  0.2960,  0.5230],
        

### Training for 1 meeting 

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

corpus_root = 'drive/My Drive/Colab Notebooks/datasets/' 

Mounted at /content/drive


In [6]:
corpus_root_DS = 'drive/My Drive/Colab Notebooks/Data Science (Final Project)/CNN/'

# tensor_dict = torch.load(corpus_root_DS+'dictionary.pt')
new_tensor_dict = torch.load(corpus_root_DS+'VAL_TENSOR_ROBERTA.pt', map_location=torch.device('cpu')) # training with cpu
# new_tensor_dict = torch.load(corpus_root_DS+'dictionary_final_lc_ELECTRA.pt')

# print(tensor_dict)
# print(len(new_tensor_dict))
# print(new_tensor_dict['Meeting 156']['Length'])

In [11]:
print(new_tensor_dict.keys())

dict_keys(['meeting: ES2011b.json', 'meeting: Bed008.json', 'meeting: IS1003b.json', 'meeting: ES2011d.json', 'meeting: ES2004c.json', 'meeting: Bmr006.json', 'meeting: TS3004a.json', 'meeting: IS1003a.json', 'meeting: education_13.json', 'meeting: covid_4.json', 'meeting: covid_9.json', 'meeting: education_9.json', 'meeting: education_4.json', 'meeting: education_17.json', 'meeting: Bmr014.json', 'meeting: TS3004d.json', 'meeting: Bed003.json', 'meeting: TS3011b.json', 'meeting: IS1003d.json', 'meeting: Bro004.json', 'meeting: TS3004b.json', 'meeting: Bro027.json', 'meeting: ES2004b.json', 'meeting: TS3004c.json', 'meeting: IS1003c.json', 'meeting: Bed016.json', 'meeting: TS3011c.json', 'meeting: ES2011c.json', 'meeting: TS3011a.json', 'meeting: TS3011d.json', 'meeting: ES2004a.json', 'meeting: ES2011a.json', 'meeting: ES2004d.json', 'meeting: Bro019.json', 'meeting: Bmr023.json'])


In [20]:
print(new_tensor_dict['meeting: Bed008.json']['Meeting 0']['Transcript'])
print(new_tensor_dict['meeting: IS1003b.json']['Meeting 0'].keys())
print(new_tensor_dict['meeting: IS1003b.json'].keys())
print(new_tensor_dict.keys())

tensor([[[-0.1095,  0.0869,  0.1253,  ...,  0.0619, -0.2003,  0.1820],
         [-0.2126, -0.0530, -0.1508,  ..., -0.0317, -0.4251, -0.0024],
         [ 0.0358,  0.2351, -0.0924,  ..., -0.3321, -0.3012, -0.0865],
         ...,
         [ 0.3888, -0.3714, -0.0690,  ...,  0.5387,  0.4168, -0.5220],
         [ 0.3570, -0.3428, -0.0045,  ...,  0.5750,  0.3514, -0.5224],
         [-0.1269,  0.0083, -0.0225,  ...,  0.1302, -0.1517,  0.0016]]],
       requires_grad=True)
dict_keys(['Length', 'Transcript', 'Query', 'Spans'])
dict_keys(['Meeting 0'])
dict_keys(['meeting: ES2011b.json', 'meeting: Bed008.json', 'meeting: IS1003b.json', 'meeting: ES2011d.json', 'meeting: ES2004c.json', 'meeting: Bmr006.json', 'meeting: TS3004a.json', 'meeting: IS1003a.json', 'meeting: education_13.json', 'meeting: covid_4.json', 'meeting: covid_9.json', 'meeting: education_9.json', 'meeting: education_4.json', 'meeting: education_17.json', 'meeting: Bmr014.json', 'meeting: TS3004d.json', 'meeting: Bed003.json', 'm

In [ ]:
print(tensor_dict['Query'][0].is_cuda)

False


### Train 1 meeting

In [ ]:
# Training test from dictionary.pt (1 meeting)
writer = SummaryWriter()

transcript_emb = tensor_dict['Transcript']
query_set = tensor_dict['Query']
start_end_label = tensor_dict['Spans'] #label

# Instantiate Locater
input_dim = 50*2  # Dimension after connected
hidden_dim = 128
output_dim = 2  # output <start, end> spans
locater = Locater(input_dim, hidden_dim, output_dim)


# Loss function
# criterion = nn.CrossEntropyLoss()  # can't work
# criterion = nn.MSELoss()  
# criterion = nn.L1Loss()
criterion = nn.SmoothL1Loss()

# Optimizer
from torch import optim
# optimizer = optim.Adam(locater.parameters(), lr=0.001)
optimizer = optim.AdamW(locater.parameters(), lr=0.001)

num_epochs = 20

for epoch in tqdm(range(num_epochs)):
  running_loss = 0.0
  for i, (inputs_query, label) in enumerate(zip(query_set, start_end_label)):
    print('i=',i)
    # print('inputs=',inputs_query)

    # 前向传播
    spans = locater(transcript_emb, inputs_query)
    loss = criterion(spans.squeeze(dim=0), label)  # 计算loss误差
    spans.requires_grad_(True)

    print('predicted spans: ', spans.squeeze(dim=0))

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
  
  
    # 打印每个epoch的平均损失
  epoch_loss = running_loss / len(query_set)
  print(f" Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss} ")
  print('======================================================')
  print()

  0%|          | 0/20 [00:00<?, ?it/s]

i= 0
predicted spans:  tensor([0.0029, 0.0693], grad_fn=<SqueezeBackward1>)
i= 1
predicted spans:  tensor([3.1264, 4.1405], grad_fn=<SqueezeBackward1>)
i= 2
predicted spans:  tensor([18.1007, 20.4746], grad_fn=<SqueezeBackward1>)
i= 3
predicted spans:  tensor([45.8711, 52.6227], grad_fn=<SqueezeBackward1>)
i= 4
predicted spans:  tensor([ 91.4300, 101.5358], grad_fn=<SqueezeBackward1>)
i= 5
predicted spans:  tensor([150.2351, 158.4020], grad_fn=<SqueezeBackward1>)


  5%|▌         | 1/20 [00:01<00:28,  1.52s/it]

 Epoch [1/20], Loss: 159.12432734171549 

i= 0
predicted spans:  tensor([277.2852, 288.8856], grad_fn=<SqueezeBackward1>)
i= 1
predicted spans:  tensor([295.0707, 304.9908], grad_fn=<SqueezeBackward1>)
i= 2
predicted spans:  tensor([262.3362, 271.5877], grad_fn=<SqueezeBackward1>)
i= 3
predicted spans:  tensor([211.1555, 222.2758], grad_fn=<SqueezeBackward1>)
i= 4
predicted spans:  tensor([166.6828, 198.1040], grad_fn=<SqueezeBackward1>)
i= 5
predicted spans:  tensor([142.9354, 182.9619], grad_fn=<SqueezeBackward1>)


 10%|█         | 2/20 [00:02<00:26,  1.48s/it]

...

 Epoch [18/20], Loss: 77.59272193908691 

i= 0
predicted spans:  tensor([  7.0631, 100.9905], grad_fn=<SqueezeBackward1>)
i= 1
predicted spans:  tensor([ 15.0573, 174.6504], grad_fn=<SqueezeBackward1>)
i= 2
predicted spans:  tensor([167.9220, 326.5852], grad_fn=<SqueezeBackward1>)
i= 3
predicted spans:  tensor([205.5013, 329.2081], grad_fn=<SqueezeBackward1>)
i= 4
predicted spans:  tensor([395.0829, 485.9073], grad_fn=<SqueezeBackward1>)
i= 5
predicted spans:  tensor([570.8956, 601.6559], grad_fn=<SqueezeBackward1>)


 95%|█████████▌| 19/20 [00:24<00:01,  1.24s/it]

 Epoch [19/20], Loss: 74.28375434875488 

i= 0
predicted spans:  tensor([12.9865, 40.0610], grad_fn=<SqueezeBackward1>)
i= 1
predicted spans:  tensor([15.4401,  0.2108], grad_fn=<SqueezeBackward1>)
i= 2
predicted spans:  tensor([52.1966, 47.7288], grad_fn=<SqueezeBackward1>)
i= 3
predicted spans:  tensor([62.9014, 38.6816], grad_fn=<SqueezeBackward1>)
i= 4
predicted spans:  tensor([193.1124, 170.7258], grad_fn=<SqueezeBackward1>)
i= 5
predicted spans:  tensor([331.5782, 300.7635], grad_fn=<SqueezeBackward1>)


100%|██████████| 20/20 [00:25<00:00,  1.28s/it]

 Epoch [20/20], Loss: 109.72569886843364 



### Train ALL meeting

In [7]:
# Training test from dictionary_final_lc.pt 

# This is for model summary(Not done yet)
writer = SummaryWriter()

# Instantiate Locater
input_dim = 2+(50*2)  # Dimension after connected
hidden_dim = 128
output_dim = 2  # output <start, end> spans
locater = Locater(input_dim, hidden_dim, output_dim)


# Loss function
# criterion = nn.CrossEntropyLoss()  # can't work
# criterion = nn.MSELoss()  
# criterion = nn.L1Loss()
criterion = nn.SmoothL1Loss()
# criterion = nn.CosineEmbeddingLoss()
# loss_range = torch.tensor([-1, 1]) # CosineEmbeddingLoss need to define range of similarity


# Optimizer
from torch import optim
# optimizer = optim.Adam(locater.parameters(), lr=0.001)
optimizer = optim.AdamW(locater.parameters(), lr=1e-5)

num_epochs = 10

for epoch in tqdm(range(num_epochs)):  # iterate epochs
  for meeting_num, meeting in new_tensor_dict.items():  # iterate meeting for an epoch

    # Get training data for one meeting
    transcript_emb = meeting['Transcript']
    query_set = meeting['Query']
    start_end_label = meeting['Spans'] #label
    meeting_len = meeting['Length'] # meeting length

    running_loss = 0.0

    for i, (inputs_query, label) in enumerate(zip(query_set, start_end_label)):  # iterate query for a meeting

      transcript_emb = transcript_emb.to('cpu')
      inputs_query = inputs_query.to('cpu')
      label = label.to('cpu')

      # 前向传播
      spans = locater(transcript_emb, inputs_query, meeting_len)
      loss = criterion(spans.squeeze(dim=0), label)  # 计算loss误差  (大部分loss function用這個)
      # loss = criterion(spans, label.unsqueeze(1), loss_range)  # 计算loss误差  (CosineEmbeddingLoss用這個)
      spans.requires_grad_(True)

      print('Meeting=',meeting_num, ' Query=',i,'  Predicted spans: ', spans.squeeze(dim=0), ' len=',meeting_len)

      # 反向传播和优化
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      running_loss += loss.item()
  
    print(" ")

    # 打印每个epoch的平均损失
  epoch_loss = running_loss / len(query_set)
  print(f" Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss} ")
  print('======================================================')
  print(" ")

  0%|          | 0/10 [00:00<?, ?it/s]

Meeting= Meeting 0  Query= 0   Predicted spans:  tensor([4.1199, 3.1364], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 1   Predicted spans:  tensor([3.6990, 3.6026], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 2   Predicted spans:  tensor([3.5947, 4.0645], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 3   Predicted spans:  tensor([3.5934, 4.4931], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 4   Predicted spans:  tensor([3.6547, 4.8937], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 5   Predicted spans:  tensor([3.7477, 5.2890], grad_fn=<SqueezeBackward1>)  len= 178
 
Meeting= Meeting 1  Query= 0   Predicted spans:  tensor([5.7288, 7.1320], grad_fn=<SqueezeBackward1>)  len= 258
Meeting= Meeting 1  Query= 1   Predicted spans:  tensor([5.8590, 7.4861], grad_fn=<SqueezeBackward1>)  len= 258
Meeting= Meeting 1  Query= 2   Predicted spans:  tensor([6.0290, 7.9056], grad_fn=<SqueezeBackward1>) 

 10%|█         | 1/10 [03:47<34:05, 227.24s/it]

Meeting= Meeting 161  Query= 11   Predicted spans:  tensor([101.7793, 118.1770], grad_fn=<SqueezeBackward1>)  len= 304
 
 Epoch [1/10], Loss: 32.864761551221214 
 
Meeting= Meeting 0  Query= 0   Predicted spans:  tensor([112.8806, 128.5631], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 1   Predicted spans:  tensor([112.3040, 128.3033], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 2   Predicted spans:  tensor([111.7395, 128.1685], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 3   Predicted spans:  tensor([112.9485, 128.4757], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 4   Predicted spans:  tensor([112.6581, 129.5703], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 5   Predicted spans:  tensor([112.7263, 130.0988], grad_fn=<SqueezeBackward1>)  len= 178
 
Meeting= Meeting 1  Query= 0   Predicted spans:  tensor([113.3842, 133.2535], grad_fn=<SqueezeBackward1>)  len= 258
Meeting= Meeting 1  Qu

 20%|██        | 2/10 [07:28<29:50, 223.81s/it]

...

 
 Epoch [8/10], Loss: 31.5982240041097 
 
Meeting= Meeting 0  Query= 0   Predicted spans:  tensor([162.4166, 228.0125], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 1   Predicted spans:  tensor([171.4716, 241.9927], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 2   Predicted spans:  tensor([163.3803, 238.6051], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 3   Predicted spans:  tensor([164.4023, 225.5741], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 4   Predicted spans:  tensor([168.1593, 240.1154], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 5   Predicted spans:  tensor([166.8724, 239.1997], grad_fn=<SqueezeBackward1>)  len= 178
 
Meeting= Meeting 1  Query= 0   Predicted spans:  tensor([164.0673, 226.7582], grad_fn=<SqueezeBackward1>)  len= 258
Meeting= Meeting 1  Query= 1   Predicted spans:  tensor([156.1952, 224.9028], grad_fn=<SqueezeBackward1>)  len= 258
Meeting= Meeting 1  Query= 

 90%|█████████ | 9/10 [32:55<03:37, 217.49s/it]

Meeting= Meeting 161  Query= 11   Predicted spans:  tensor([ 98.8552, 122.4120], grad_fn=<SqueezeBackward1>)  len= 304
 
 Epoch [9/10], Loss: 31.31916904449463 
 
Meeting= Meeting 0  Query= 0   Predicted spans:  tensor([165.7161, 234.5899], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 1   Predicted spans:  tensor([176.5777, 251.4177], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 2   Predicted spans:  tensor([167.5720, 246.7949], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 3   Predicted spans:  tensor([167.3930, 232.2900], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 4   Predicted spans:  tensor([172.2281, 248.8180], grad_fn=<SqueezeBackward1>)  len= 178
Meeting= Meeting 0  Query= 5   Predicted spans:  tensor([170.8055, 247.2946], grad_fn=<SqueezeBackward1>)  len= 178
 
Meeting= Meeting 1  Query= 0   Predicted spans:  tensor([167.4520, 233.9067], grad_fn=<SqueezeBackward1>)  len= 258
Meeting= Meeting 1  Que

100%|██████████| 10/10 [36:36<00:00, 219.61s/it]

 
 Epoch [10/10], Loss: 30.81246344248454 
 


In [ ]:
# Evaluate the performance of Locater


In [8]:
# Save the Locater
torch.save(locater, corpus_root_DS+'locater(ELECTRA).pth')

### Evaluate the Locator

In [7]:
locater = torch.load(corpus_root_DS+'locater(RoBERTa).pth')

In [ ]:
# Evaluate the performance of Locater (with train set)

cos_sim_value = []
manhattan_dis = []
minkowski_dis = []

for meeting_num, meeting in new_tensor_dict.items():  # iterate meeting
   # Get training data for one meeting
    transcript_emb = meeting['Transcript']
    query_set = meeting['Query']
    start_end_label = meeting['Spans'] #label
    meeting_len = meeting['Length'] # meeting length

    for i, (inputs_query, label) in enumerate(zip(query_set, start_end_label)):  # iterate query for a meeting

      transcript_emb = transcript_emb.to('cpu')
      inputs_query = inputs_query.to('cpu')
      label = label.to('cpu')

      predicted_span = locater(transcript_emb, inputs_query, meeting_len)

      # Cosine similarity
      cos_sim = F.cosine_similarity(predicted_span, label)

      # Manhattan Distance
      manhattan_distance = torch.sum(torch.abs(predicted_span - label))

      # Minkowski Distance (p=1.5)
      minkowski_distance = F.pairwise_distance(predicted_span.unsqueeze(0), label.unsqueeze(0), p=1.5)

      print('Meeting ', meeting_num, 'Query ', i, 'len=', meeting_len)
      print('predicted_span=', predicted_span)
      print('')
      print('cos_sim=', cos_sim)
      print('Manhattan dis=', manhattan_distance)
      print('Minkowski dis=', minkowski_distance)
      print('')

      cos_sim_value.append(cos_sim) # cos_sim of all predicted_value & true value
      manhattan_dis.append(manhattan_distance) # manhattan of all predicted_value & true value
      minkowski_dis.append(minkowski_distance) # minkowski of all predicted_value & true value
  
    print('==============================')

print('Average cosine similarity for prediction =', (sum(cos_sim_value)/len(cos_sim_value)))
print('Average Manhattan distance for prediction =', (sum(manhattan_dis)/len(manhattan_dis)))
print('Average Minkowski distance for prediction =', (sum(minkowski_distance)/len(minkowski_distance)))

In [11]:
# Evaluate the performance of Locater (with validation set)

cos_sim_value = []
manhattan_dis = []
minkowski_dis = []
euclid_dis = []

for meeting_num, meeting in new_tensor_dict.items():  # iterate meeting
   # Get training data for one meeting
    transcript_emb = meeting['Meeting 0']['Transcript']
    query_set = meeting['Meeting 0']['Query']
    start_end_label = meeting['Meeting 0']['Spans'] #label
    meeting_len = meeting['Meeting 0']['Length'] # meeting length

    for i, (inputs_query, label) in enumerate(zip(query_set, start_end_label)):  # iterate query for a meeting

      transcript_emb = transcript_emb.to('cpu')
      inputs_query = inputs_query.to('cpu')
      label = label.to('cpu')

      predicted_span = locater(transcript_emb, inputs_query, meeting_len)

      # Cosine similarity
      cos_sim = F.cosine_similarity(predicted_span, label)

      # Manhattan Distance
      manhattan_distance = torch.sum(torch.abs(predicted_span - label))

      # Minkowski Distance (p=1.5)
      minkowski_distance = F.pairwise_distance(predicted_span.unsqueeze(0), label.unsqueeze(0), p=1.5)

      # Euclid Distance
      euclid = torch.sqrt(torch.sum((predicted_span - label) ** 2, dim=1))

      print('Meeting ', meeting_num, 'Query ', i, 'len=', meeting_len)
      print('predicted_span=', predicted_span, 'gold_span=', label)
      print('Euclid distance=', euclid)
      print('cos_sim=', cos_sim)
      print('Manhattan dis=', manhattan_distance)
      print('Minkowski dis=', minkowski_distance)
      print('')

      euclid_dis.append(euclid)  # Euclid of all predicted_value & true value
      cos_sim_value.append(cos_sim) # cos_sim of all predicted_value & true value
      manhattan_dis.append(manhattan_distance) # manhattan of all predicted_value & true value
      minkowski_dis.append(minkowski_distance) # minkowski of all predicted_value & true value
  
    print('==============================')

print('Average Euclid Distance for prediction =', (sum(euclid_dis)/len(euclid_dis)))
print('Average cosine similarity for prediction =', (sum(cos_sim_value)/len(cos_sim_value)))
print('Average Manhattan distance for prediction =', (sum(manhattan_dis)/len(manhattan_dis)))
print('Average Minkowski distance for prediction =', (sum(minkowski_distance)/len(minkowski_distance)))

Meeting  meeting: ES2011b.json Query  0 len= 361
predicted_span= tensor([[217.3151, 280.9408]], grad_fn=<AbsBackward0>) gold_span= tensor([ 1., 52.], requires_grad=True)
Euclid distance= tensor([314.9701], grad_fn=<SqrtBackward0>)
cos_sim= tensor([0.8026], grad_fn=<SumBackward1>)
Manhattan dis= tensor(445.2560, grad_fn=<SumBackward0>)
Minkowski dis= tensor([[353.4710]], grad_fn=<NormBackward1>)

Meeting  meeting: ES2011b.json Query  1 len= 361
predicted_span= tensor([[213.4664, 275.1346]], grad_fn=<AbsBackward0>) gold_span= tensor([73., 88.], requires_grad=True)
Euclid distance= tensor([233.9875], grad_fn=<SqrtBackward0>)
cos_sim= tensor([0.9995], grad_fn=<SumBackward1>)
Manhattan dis= tensor(327.6010, grad_fn=<SumBackward0>)
Minkowski dis= tensor([[261.3363]], grad_fn=<NormBackward1>)

Meeting  meeting: ES2011b.json Query  2 len= 361
predicted_span= tensor([[215.0727, 269.6426]], grad_fn=<AbsBackward0>) gold_span= tensor([ 89., 120.], requires_grad=True)
Euclid distance= tensor([195.6